In [2]:
import os 
import sys
import json     #######我的这个项目需要这个东西吗#########
import torch
import torch.nn as nn
from torchvision import transforms, datasets 
import torch.optim as optim 
from tqdm import tqdm  
from classic_models.alexnet import AlexNet

In [15]:
! pip install torchsummary
 

In [17]:
# 判断可用设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))

using cpu device.


In [18]:
# 注意改成自己的数据集路径
data_path = "D:/Machine Learning/点头课程内容/22.05.22/AlexNet(1)/AlexNet/flower"
assert os.path.exists(data_path), "{} path does not exist.".format(data_path)

AssertionError: D:/Machine Learning/点头课程内容/22.05.22/AlexNet(1)/AlexNet/flower path does not exist.

In [ ]:
 # 数据预处理与增强
""" 
ToTensor()能够把灰度范围从0-255变换到0-1之间的张量.
transform.Normalize()则把0-1变换到(-1,1). 具体地说, 对每个通道而言, Normalize执行以下操作: image=(image-mean)/std
其中mean和std分别通过(0.5,0.5,0.5)和(0.5,0.5,0.5)进行指定。原来的0-1最小值0则变成(0-0.5)/0.5=-1; 而最大值1则变成(1-0.5)/0.5=1. 
也就是一个均值为0, 方差为1的正态分布. 这样的数据输入格式可以使神经网络更快收敛。
"""
data_transform = {
    "train": transforms.Compose([transforms.Resize(224),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),

    "val": transforms.Compose([transforms.Resize((224, 224)),  # val不需要任何数据增强
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])}

In [ ]:
# 使用ImageFlolder加载数据集中的图像，并使用指定的预处理操作来处理图像， ImageFlolder会同时返回图像和对应的标签。 (image path, class_index) tuples
train_dataset = datasets.ImageFolder(root=os.path.join(data_path, "train"), transform=data_transform["train"])
validate_dataset = datasets.ImageFolder(root=os.path.join(data_path, "val"), transform=data_transform["val"])
train_num = len(train_dataset)
val_num = len(validate_dataset)

In [ ]:
# 使用class_to_idx给类别一个index，作为训练时的标签： {'daisy':0, 'dandelion':1, 'roses':2, 'sunflower':3, 'tulips':4}
flower_list = train_dataset.class_to_idx
flower_list

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}

In [2]:
# 创建一个字典，存储index和类别的对应关系，在模型推理阶段会用到。
cla_dict = dict((val, key) for key, val in flower_list.items())
cla_dict

NameError: name 'flower_list' is not defined

In [3]:
# 将字典写成一个json文件
json_str = json.dumps(cla_dict, indent=4)
with open( os.path.join(data_path, 'class_indices.json') , 'w') as json_file:
    json_file.write(json_str)
json_str

NameError: name 'cla_dict' is not defined

In [4]:
batch_size = 64 # batch_size大小，是超参，可调，如果模型跑不起来，尝试调小batch_size
num_workers = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # 用于加载数据集的进程数量
print('Using {} dataloader workers every process'.format(num_workers))

Using 8 dataloader workers every process


In [5]:
# 使用 DataLoader 将 ImageFloder 加载的数据集处理成批量（batch）加载模式
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
validate_loader = torch.utils.data.DataLoader(validate_dataset, batch_size=4, shuffle=False,  num_workers=num_workers) # 注意，验证集不需要shuffle
print("using {} images for training, {} images for validation.".format(train_num, val_num))

NameError: name 'train_dataset' is not defined

In [6]:
# 实例化模型，并送进设备
net = AlexNet(num_classes=5, init_weights=True)
net.to(device)

NameError: name 'AlexNet' is not defined

In [7]:
 # 指定损失函数用于计算损失；指定优化器用于更新模型参数；指定训练迭代的轮数，训练权重的存储地址
loss_function = nn.CrossEntropyLoss() 
optimizer = optim.Adam(net.parameters(), lr=0.0002)
epochs = 70
save_path = os.path.abspath(os.path.join(os.getcwd(), './results/weights/alexnet'))   #os.getcwd()返回当前目录
if not os.path.exists(save_path):    
    os.mkdir(save_path)         #与Linux下相同语法

best_acc = 0.0 # 初始化验证集上最好的准确率，以便后面用该指标筛选模型最优参数。
train_steps = len(train_loader) # epoch是迭代训练数据集的次数，train_steps是数据集可以被分成的批次数量 = num(dataset) / batch_size

NameError: name 'net' is not defined

In [8]:
for epoch in range(epochs):
        ############################################################## train ######################################################
        net.train()
        train_loss = torch.zeros(1).to(device) # 初始化，用于计算训练损失           torch.zeros(2, 3)————这就是torch.zeros的用法，括号内是size
                                                                            # tensor([[ 0.,  0.,  0.],
                                                                            #         [ 0.,  0.,  0.]])
        acc_num = torch.zeros(1).to(device)    # 初始化，用于计算训练过程中预测正确的数量
        sample_num = 0                         # 初始化，用于记录当前迭代中，已经计算了多少个样本
        # tqdm是一个进度条显示器，可以在终端打印出现在的训练进度
        train_bar = tqdm(train_loader, file=sys.stdout, ncols=100)   #sys.stdout还是不太懂，执行一下代码看看了
        for step, data in enumerate(train_bar):         #？？？train_bar是什么信息
            images, labels = data
            sample_num += images.shape[0]
            optimizer.zero_grad()
            outputs = net(images.to(device))          # output_shape: [batch_size, num_classes]   这里的images应该是前向传播forward中的x
            pred_class = torch.max(outputs, dim=1)[1] # torch.max 返回值是一个tuple，第一个元素是max值，第二个元素是max值的索引。  
            #这里dim表示要降维的维度，pred_class范围的是分类号
            acc_num += torch.eq(pred_class, labels.to(device)).sum() #torch.eq()判断后面两个数组对应元素是否相等，相等为true，不等为flase，这里对bool数组求和啥意思呢

            loss = loss_function(outputs, labels.to(device)) # 求损失，  ？？？为什么上下同时对labels做计算，但是计算的维度信息并不匹配啊？？？
            loss.backward() # 自动求导
            optimizer.step() # 梯度下降

            # print statistics
            train_loss += loss.detach()  / (step + 1)
            train_acc = acc_num.item() / sample_num 
            # .desc是进度条tqdm中的成员变量，作用是描述信息
            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1, epochs, loss)

        # validate
        net.eval()
        acc_num = 0.0  # accumulate accurate number per epoch
        with torch.no_grad(): 
            for val_data in validate_loader:
                val_images, val_labels = val_data
                outputs = net(val_images.to(device))
                predict_y = torch.max(outputs, dim=1)[1]
                acc_num += torch.eq(predict_y, val_labels.to(device)).sum().item() 

        val_accurate = acc_num / val_num
        print('[epoch %d] train_loss: %.3f  train_acc: %.3f  val_accuracy: %.3f' %  (epoch + 1, train_loss / train_steps , train_acc, val_accurate))   
        # 判断当前验证集的准确率是否是最大的，如果是，则更新之前保存的权重
        if val_accurate > best_acc:
            best_acc = val_accurate
            torch.save(net.state_dict(), os.path.join(save_path, "AlexNet.pth") )

        # 每次迭代后清空这些指标，重新计算
        train_loss = 0.0
        train_acc = 0.0
        val_accurate = 0.0

NameError: name 'epochs' is not defined

In [9]:
pred_class

NameError: name 'pred_class' is not defined

In [10]:
labels.shape

NameError: name 'labels' is not defined

In [11]:
images.shape

NameError: name 'images' is not defined

In [12]:
outputs = net(images.to(device))
outputs.shape

NameError: name 'net' is not defined

In [13]:
pred_class = torch.max(outputs, dim=1)[1]
pred_class.shape

NameError: name 'outputs' is not defined